In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker
from google.cloud import bigquery
import os

!pip install openpyxl ## read excel files
!pip install wrds
import wrds

print('----------------\n Establishing WRDS connection:')
# establish connection
conn = wrds.Connection()



----------------
 Establishing WRDS connection:


Enter your WRDS username [jupyter]: rogermasclans
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [9]:

# comp_global_daily.g_secd 
stock_data = conn.raw_sql(
f"""

SELECT
    gvkey, cik, conm
FROM comp_global_daily.g_company
where gvkey = '001166'
limit 10
"""
)
stock_data
# WHERE comp_global_daily.g_secd.datadate BETWEEN
    # '2024-01-01'::date AND '2024-01-05'::date
# 1999-02-05 where gvkey = '17095' 

stock_data = conn.raw_sql(
f"""

SELECT
    gvkey, cik, conm, fic
FROM comp_global_daily.g_company
where gvkey = '001166'
limit 10
"""
)
stock_data

# https://wrds-www.wharton.upenn.edu/data-dictionary/comp_global_daily/g_company/


,gvkey,cik,conm,fic
0,001166,0000351483,ASM INTERNATIONAL NV,NLD


In [3]:
stock_data.columns

Index(['gvkey', 'iid', 'datadate', 'conm', 'curcdd', 'ajexdi', 'cshoc',
       'cshtrd', 'prccd', 'prchd', 'prcld', 'prcod', 'prcstd', 'qunit',
       'curcddv', 'cheqv', 'cheqvgross', 'cheqvnet', 'cheqvtm', 'div', 'divd',
       'divdgross', 'divdnet', 'divdtm', 'divgross', 'divnet', 'divrc',
       'divrcgross', 'divrcnet', 'divsp', 'divspgross', 'divspnet', 'divsptm',
       'anncdate', 'cheqvpaydate', 'divdpaydate', 'divrcpaydate',
       'divsppaydate', 'paydate', 'recorddate', 'split', 'splitf', 'trfd',
       'epf', 'exchg', 'isin', 'secstat', 'sedol', 'tpci', 'fic', 'gind',
       'gsubind', 'loc', 'monthend'],
      dtype='object')

In [7]:

####SET BG PROJECT TO FETCH DATA
# Define project on BQ
project_id = "starfire-410116"

# Authenticate to Google Cloud
client = bigquery.Client(project=project_id)

# Set allow_large_results to Truepoti
job_config = bigquery.QueryJobConfig()
job_config.allow_large_results = True

dataset_id = 'derived'
table_id = 'stock_index_data_1980_1999'

# Authenticate with Google Cloud using your service account key
client = bigquery.Client(project=project_id)

# Specify the BigQuery table reference
table_ref = client.dataset(dataset_id).table(table_id)

# Upload the Pandas DataFrame to BigQuery with auto-detection
job_config = bigquery.LoadJobConfig(
  autodetect=True, # Enable auto-detection of the schema
  write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE # You can choose your write disposition
)

# Upload the DataFrame to BigQuery
job = client.load_table_from_dataframe(stock_index_data, table_ref, job_config=job_config)
job.result() # Wait for the job to complete
print(f'... done')


/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/_pandas_helpers.py:537: UserWarning: Pyarrow could not determine the type of columns: dvpsxd, newnum, oldnum, prccddiv, prccddivn, prchd, prcld.
  warnings.warn(


... done
